# **Curso de Machine Learning - IA Expert Academy**


---


**Alunos:** Lucas Dias Noronha

# **Combinação de Classificadores**

A combinação de classificadores é uma técnica que visa melhorar o desempenho de um sistema de classificação, combinando as previsões de vários classificadores individuais. Essa abordagem pode ajudar a mitigar as fraquezas de cada classificador individual e aproveitar suas respectivas forças.

Existem várias maneiras de combinar classificadores, sendo as mais comuns:

Votação por maioria: Cada classificador vota em uma classe e a classe mais frequente é selecionada como a previsão final.

Votação ponderada: Cada classificador atribui um peso à sua previsão, levando em consideração a confiança ou desempenho do classificador. As previsões são ponderadas e a classe com maior soma ponderada é selecionada.

Empilhamento (stacking): Os classificadores são organizados em um esquema hierárquico, onde as previsões de um conjunto de classificadores são usadas como entrada para outro classificador, que faz a previsão final.

Combinação por média: As previsões dos classificadores são combinadas calculando-se a média das probabilidades de cada classe.

Boosting: Os classificadores são treinados sequencialmente, onde cada classificador subsequente é treinado para corrigir as previsões erradas dos classificadores anteriores. As previsões de todos os classificadores são ponderadas e combinadas para obter a previsão final.

A escolha da técnica de combinação de classificadores depende do contexto e das características dos dados. É importante considerar a diversidade dos classificadores individuais e suas habilidades de generalização. Além disso, ajustar os pesos atribuídos a cada classificador pode ajudar a melhorar o desempenho geral da combinação.

A combinação de classificadores é amplamente utilizada em diversas áreas, como reconhecimento de padrões, aprendizado de máquina e classificação de dados em geral.

In [4]:
import pickle
import numpy as np

## **Carregando Modelos**

In [44]:
# Carregand modelo Rede Neural.
rede_neural = pickle.load(open('rede_neural_finalizado.sav', 'rb'))

# Carregando modelo Árvore de Decisão.
arvore = pickle.load(open('arvore_finalizado.sav', 'rb'))

# Carregando modelo SVM.
svm = pickle.load(open('svm_finalizado.sav', 'rb'))

## **Preparando os Dados**

In [46]:
# Carregando dados de treinamento para simular uma situação real.
with open('credit.pkl', 'rb') as f:
  X_credit_treinamento, y_credit_treinamento, X_credit_teste, y_credit_teste = pickle.load(f)

In [47]:
# Juntando os dados.
X_credit = np.concatenate((X_credit_treinamento, X_credit_teste), axis = 0)
y_credit = np.concatenate((y_credit_treinamento, y_credit_teste), axis = 0)

In [48]:
X_credit.shape, y_credit.shape # Retorna linhas e colunas.

((2000, 3), (2000,))

## **Aplicando os modelos**

In [49]:
# Acessando registro e preparando-o para ser usado como simulação de dados de um cliente real.
novo_registro = X_credit [1999]
novo_registro = novo_registro.reshape(1, -1)
novo_registro, novo_registro.shape

(array([[-1.03572293, -0.93978122,  0.04244312]]), (1, 3))

In [50]:
# Armazenando a previsão do modelo Rede Neural.
resposta_rede_neural = rede_neural.predict(novo_registro)

# Armazenando a previsão do modelo Árvore de Decisão.
resposta_arvore = arvore.predict(novo_registro)

# Armazenando a previsão do modelo de Árvore de Decisão.
resposta_svm = svm.predict(novo_registro)

In [51]:
# Previsão 1 para todos os modelos, que indica que o cliente não pagará o empréstimo.
resposta_rede_neural[0], resposta_arvore[0], resposta_svm[0]

(1, 1, 1)

## **Aplicando a combinação de classificadores utilizando a técnica de votação por maioria**

In [52]:
# Criando um tupla que armazerá as repostas de cada modelo.
classificador_votos = [
    resposta_rede_neural,
    resposta_arvore,
    resposta_svm
]

# Conta a quantidade de vezes que a classe "paga" (0) e a classe "não paga" (1) aparecem na lista.
# Ele também armazenará a contagem de cada classe em suas respectivas variáveis.
paga = classificador_votos.count(0)
n_paga = classificador_votos.count(1)

# A decisão final será baseada na votação por maioria.
if paga > n_paga:
    print('Previsão: O cliente provavelmente pagará o empréstimo.')
elif paga == n_paga:
    print('Previsão: Há um empate na decisão de pagamento do empréstimo.')
else:
    print('Previsão: O cliente provavelmente não pagará o empréstimo.')

Previsão: O cliente provavelmente não pagará o empréstimo.


# **Rejeição de Classificadores**

A rejeição de classificadores é uma técnica utilizada quando um classificador não tem confiança suficiente em sua previsão ou quando os dados de entrada estão fora da distribuição conhecida. Em vez de fazer uma previsão incorreta, o classificador opta por rejeitar a amostra e não fornecer uma classificação.

A rejeição de classificadores pode ser útil em situações em que é preferível não tomar uma decisão incorreta ou quando a incerteza é alta. Em vez de fornecer uma previsão arriscada, o classificador pode indicar que não possui confiança suficiente para fazer uma previsão precisa.

Para implementar a rejeição de classificadores, podemos adicionar uma condição que verifica a confiança da previsão ou outros critérios relevantes. Se o classificador não atender aos critérios de confiança, a amostra é rejeitada.

In [66]:
# Acessando registro e preparando-o para ser usado como simulação de dados de um cliente real.
novo_registro = X_credit [0]
novo_registro = novo_registro.reshape(1, -1)
novo_registro, novo_registro.shape

(array([[-1.3754462 ,  0.50631087,  0.10980934]]), (1, 3))

In [67]:
# Armazenando a previsão do modelo Rede Neural.
resposta_rede_neural = rede_neural.predict(novo_registro)

# Armazenando a previsão do modelo Árvore de Decisão.
resposta_arvore = arvore.predict(novo_registro)

# Armazenando a previsão do modelo de Árvore de Decisão.
resposta_svm = svm.predict(novo_registro)

In [68]:
# Previsão 1 para todos os modelos, que indica que o cliente não pagará o empréstimo.
resposta_rede_neural[0], resposta_arvore[0], resposta_svm[0]

(0, 0, 0)

In [69]:
# Estimando as probabilidades das classes para cada modelo.
probabilidade_rede_neural = rede_neural.predict_proba(novo_registro)
probabilidade_arvore = arvore.predict_proba(novo_registro)
probabilidade_svm = svm.predict_proba(novo_registro)

In [70]:
# Retornando a probabilidade de cada classe para cada modelo.
probabilidade_rede_neural, probabilidade_arvore, probabilidade_svm

(array([[1.00000000e+00, 4.48941181e-17]]),
 array([[1., 0.]]),
 array([[9.99997789e-01, 2.21110142e-06]]))

In [71]:
# Calculando a confiança máxima do modelo Rede Neural.
# Obtendo o valor máximo da lista de probabilidades (classe com maior probabilidade) para o modelo Rede Neural.
confianca_rede_neural = probabilidade_rede_neural.max()

# Retornando confiança.
confianca_rede_neural

1.0

In [63]:
# Calculando a confiança máxima do modelo Árvore de Decisão.
# Obtendo o valor máximo.
confianca_arvore = probabilidade_arvore.max()

# Retornando confiança.
confianca_arvore

1.0

In [72]:
# Calculando a confiança máxima do modelo SVM.
# Obtendo o valor máximo.
confianca_svm = probabilidade_svm.max()

# Rertoando a confiança.
confianca_svm

0.9999977888985752

In [76]:
# Função para imprimir a decisão final.
def imprimir_decisao(tem_confianca, paga, n_paga, algoritmo):
    # Verifica se alguma previsão foi realizada.
    if tem_confianca:
        # Compara as contagens para determinar a decisão final.
        if paga > n_paga:
            print(f'Baseado na análise de {algoritmo} algoritmos, é previsto que o cliente pagará o empréstimo.')
        elif paga == n_paga:
            print(f'Baseado na análise de {algoritmo} algoritmos, há um empate na decisão.')
        else:
            print(f'Baseado na análise de {algoritmo} algoritmos, é previsto que o cliente não pagará o empréstimo.')
    else:
        print('O algoritmo não foi capaz de fazer uma previsão.')

# Criando um tupla que armazerá as repostas de cada modelo.
classificador_votos = [
    resposta_rede_neural,
    resposta_arvore,
    resposta_svm
]

# Conta a quantidade de vezes que a classe "paga" (0) e a classe "não paga" (1) aparecem na lista.
# Ele também armazenará a contagem de cada classe em suas respectivas variáveis.
paga = classificador_votos.count(0)
n_paga = classificador_votos.count(1)

# Confiança mínima.
confianca_minima = 0.999999

# Contador algoritmo.
algoritmo = 0

# Se possuir confiança mínima, será atribuido o valor True.
tem_confianca = False

# Verifica se a confiança da Rede Neural é maior ou igual à confiança mínima.
if confianca_rede_neural >= confianca_minima:
    # Incrementa o número de algoritmos utilizados.
    algoritmo += 1
    # Indica que uma previsão foi realizada.
    tem_confianca = True

# Repete o mesmo processo para a Árvore de Decisão.
if confianca_arvore >= confianca_minima:
    algoritmo += 1
    tem_confianca = True

# Repete o mesmo processo para o SVM.
if confianca_svm >= confianca_minima:
    algoritmo += 1
    tem_confianca = True

# Chama a função para imprimir a decisão final
imprimir_decisao(tem_confianca, paga, n_paga, algoritmo)



Baseado na análise de 2 algoritmos, é previsto que o cliente pagará o empréstimo.


# **Considerações Finais**

Neste estudo de caso, tive a oportunidade de aprofundar meu conhecimento sobre as poderosas técnicas de Combinação de Classificadores e Rejeição de Classificadores. Durante a exploração dessas abordagens, pude compreender como elas podem ser aplicadas para melhorar a precisão e confiabilidade dos resultados de classificação.

A técnica de Combinação de Classificadores permitiu a integração de múltiplos classificadores individuais, cada um com suas próprias forças e fraquezas, para chegar a uma decisão final mais robusta e precisa. Ao ponderar as previsões dos classificadores com base na técnica de votação por maioria, pude obter uma visão mais abrangente dos dados e aproveitar o conhecimento coletivo dos modelos.

Por outro lado, a técnica de Rejeição de Classificadores mostrou-se valiosa para a incerteza ou a falta de confiança em uma previsão quando precisamos que precisão seja bastante alta. Ao estabelecer critérios para rejeitar amostras quando a confiança era baixa ou quando os dados estavam fora da distribuição conhecida, podemos evitar tomar decisões incorretas ou arriscadas. Isso resulta em uma abordagem mais cautelosa e confiável na análise e classificação de dados.

Essas técnicas, em conjunto, representam ferramentas poderosas para melhorar a precisão e a confiabilidade dos sistemas de classificação. Ao explorar e compreender os princípios por trás da Combinação de Classificadores e da Rejeição de Classificadores, estou preparado para aplicá-las de forma eficaz em futuros estudos e projetos que envolvam análise e classificação de dados.

Agradeço ao professor Jones Granatyr pelo conhecimento compartilhado e pela orientação durante o curso. Essa experiência contribuiu significativamente para minha compreensão de algoritmos de Machine Learning e sua aplicação prática.